In [1]:
import os

if 'main_dir' in globals():
    os.chdir(main_dir)

import pandas as pd
from utils.load_config import (
    load_exp_config, 
    get_value_from_fields, 
    get_df_historical_data,
    get_list_distorical_data
    )

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
# plt.style.use("tableau-colorblind10")
plt.style.use("seaborn-v0_8-whitegrid")

from regelum.objective import RunningObjective
from regelum.model import ModelQuadLin
import numpy as np


if os.getcwd().endswith("notebooks"):
    main_dir = os.getcwd()
    os.chdir("..")

In [2]:
try:
    from src.objective import ThreeWheeledRobotCostWithSpot
except:
    from src.objective import ThreeWheeledRobotCostWithSpot

In [3]:

objective_function = RunningObjective(ModelQuadLin(
    weights= [2, 10, 0, 0, 0],
    quad_matrix_type="diagonal",
    is_with_linear_terms=False,
))

objective_function = ThreeWheeledRobotCostWithSpot(
    quadratic_model=ModelQuadLin(
        weights= [20, 100, 0, 0, 0],
        quad_matrix_type="diagonal",
        is_with_linear_terms=False),
        spot_gain=100,
        spot_x_center=-0.5,
        spot_y_center=-0.5,
        spot_std=0.2,
    )


def get_sampling_time(exp_path):
    config = load_exp_config(exp_path)

    sampling_time = config['scenario']['sampling_time']
    if "$" in sampling_time:
        correct_fields = sampling_time.strip("\{\}$").split(".")
        sampling_time = get_value_from_fields(config, correct_fields)

    return sampling_time

def cal_obj_df(row):
    observation = np.expand_dims(np.array(row.loc[["x [m]", "y [m]", "angle [rad]"]].values), axis=0)
    action = np.expand_dims(np.array(row.loc[["velocity [m/s]", "angular velocity [rad/s]"]].values), axis=0)
    
    return objective_function(observation, action)

In [4]:
class ExperimentData():
    def __init__(self, exp_path, chosen_name, **kwarg) -> None:
        self.df = get_df_historical_data(exp_path, chosen_name)
        self.sampling_time = get_sampling_time(exp_path)
        self.color = kwarg.get("color")
        
        self.correct_column_name()
        self.calculate_objective_values()

    def correct_column_name(self):
        replacements = {
            "x_rob"   :     "x [m]", 
            "y_rob"   :    "y [m]",
            "vartheta":       "angle [rad]",
            "v"       : "velocity [m/s]",
            "omega"   :    "angular velocity [rad/s]",
        }

        new_columns = []
        if "x [m]" not in self.df.columns.values:
            for col in self.df.columns.values:
                new_columns.append(col if col not in replacements else replacements[col])

            self.df.columns = new_columns

    def calculate_objective_values(self):
        self.df["objective_value"] = self.df.apply(cal_obj_df, axis=1)
        self.df["accumulative_objective"] = self.df["objective_value"].apply(lambda x: x*self.sampling_time).cumsum()

    def plot_cost(self, ax, exp_name="", with_others=False):
        titles = [
            "Objective value",
            "Accumulative objective value"
        ]
        targets = [
            "objective_value",
            "accumulative_objective"
        ]
        if not with_others:
            titles[0] += "of {} \n final cost: {:0.2f}".format(exp_name, self.df.accumulative_objective.max())
            titles[1] += "of {} \n final cost: {:0.2f}".format(exp_name, self.df.accumulative_objective.max())

        for i in range(len(titles)):
            ax[i].set_title(titles[i])
            self.df.plot(x="time", y=targets[i], ax=ax[i], label=exp_name if with_others else None, alpha=0.8, color=self.color)
            ax[i].grid()

    def plot_trajectory(self, ax, exp_name=""):
        ax.set_title("Trajectory")
        ax.plot(self.df["x [m]"], self.df["y [m]"], label=exp_name, alpha=0.8, color=self.color)
        ax.grid()
    

In [5]:
experiment_collections = {
    "CALF controller": {
        "exp_path": "/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-20/15-51-23/0",
        "chosen_name": "observations_actions_it_00001"
    },
    "MPC controller": {
        "exp_path": "/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-20/15-53-14/0",
        "chosen_name": "observations_actions_it_00001"
    },
    "SARSA_m controller": {
        "exp_path": "/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-20/15-56-16/0",
        "chosen_name": "observations_actions_it_00001"
    },
    "Nominal controller": {
        "exp_path": "/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-20/15-59-31/0",
        "chosen_name": "observations_actions_it_00001"
    },
    "PPO controller": {
        "exp_path": "/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-20/16-01-28/0",
        "chosen_name": "observations_actions_it_00001"
    },
}

cm = plt.get_cmap("tab10")

color_values = np.linspace(0, 1, 10)

for idx, exp in enumerate(experiment_collections):
    experiment_collections[exp]["color"] = cm(color_values[idx])
    experiment_collections[exp]["obj"] = ExperimentData(**experiment_collections[exp])


## Re-calculate objective

### Define objective function

In [6]:

observation = np.array([[1, 1, 0.01]])
actions = np.array([[1, 1]])
objective_function(observation, actions)

120.0

## Plot objective

In [7]:
plt.clf()
fig, ax = plt.subplots(1, 2, figsize=(11, 5))

for id, exp in enumerate(experiment_collections):
    experiment_collections[exp]["obj"].plot_cost(ax, exp, with_others=True)

for _ax in ax:
    _ax.grid()
# ppo_exp.plot_cost(ax[0], "PPO Controller")
# nominal_exp.plot_cost(ax[1], "Nominal Controller")

In [8]:
x_lim = y_lim = [-1.7, 0.3]
X = np.arange(x_lim[0], x_lim[1], 0.01)
Y = np.arange(y_lim[0], y_lim[1], 0.01)

X, Y = np.meshgrid(X, Y)
Z = []
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z.append(objective_function(observation=np.array([[X[i, j], Y[i, j], 0]]),
                                    action=np.array([[0, 0]])))

Z = np.array(Z).reshape((X.shape[0], X.shape[1]))

In [9]:
plt.clf()
fig, ax = plt.subplots()

plt.figure(facecolor='white')  # Set figure background color
plt.rcParams.update({"figure.figsize" : (12, 8),
                     "axes.facecolor" : "white",
                     "axes.edgecolor":  "black"})

cs = ax.contourf(X, Y, Z, alpha=0.8, levels=35, cmap="BuPu")
cs.cmap.set_over('red')
cs.cmap.set_under('blue')
cs.changed()

# Make a colorbar for the ContourSet returned by the contourf call.
cbar = fig.colorbar(cs)
cbar.ax.set_ylabel('Cost')

for id, exp in enumerate(experiment_collections):
    experiment_collections[exp]["obj"].plot_trajectory(ax, exp)

ax.annotate(text="Target", xy=(0, 0), ha='center', va='center')
goal_circle = plt.Circle((0, 0), 0.1, color="yellowgreen")
ax.add_artist(goal_circle)

ax.grid()
ax.legend()

In [10]:
np.min(Z), np.max(Z)
Z = np.where(Z<1,np.zeros_like(Z), Z)

# Check log to plot Cost in a CI

In [11]:
import os
from datetime import datetime
import pathlib

In [12]:
ROOT_DIR = "./regelum_data/outputs/"

start_datetime_str = "2024-08-21 16-45-08"
end_datetime_str = "2024-08-22 16-45-08"
date_format = '%Y-%m-%d %H-%M-%S'

start_date_time = datetime.strptime(start_datetime_str, date_format)
end_date_time = datetime.strptime(end_datetime_str, date_format)
date_folder = os.listdir(ROOT_DIR)

valid_paths = []
for d in date_folder:
    for t in os.listdir(os.path.join(ROOT_DIR, d)):
        tmp_datetime = datetime.strptime(f"{d} {t}", date_format)
        if tmp_datetime < start_date_time or end_date_time < tmp_datetime:
            continue

        valid_paths.append(str(pathlib.Path(os.path.join(ROOT_DIR, d, t)).absolute()))

In [14]:
path_hierachy = {}
for p in valid_paths:
    path_hierachy[p] = get_list_distorical_data(p)

path_hierachy

{
    '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37': [
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00049.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00060.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00025.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00118.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00094.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00122.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00017.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00061.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00024.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00090.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00116.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00009.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00093.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00045.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00097.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00075.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00029.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00041.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00048.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00033.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00077.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00036.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00038.h5',
        '/home/robosrv/huyhoang/regelum-playground/regelum_data/outputs/2024-08-21/20-58-37/0/.callbacks/HistoricalDataCallback/observations_actions_it_00095

In [32]:
total_dfs = []
for exp_path in path_hierachy:
    exp_dfs = []
    for iteration_path in path_hierachy[exp_path]:
        exp_dfs.append(get_df_historical_data(absolute_path=iteration_path))
        exp_dfs[-1]["absolute_path"] = iteration_path
    
    if len(exp_dfs) == 0:
        continue
    
    exp_df = pd.concat(exp_dfs)
    exp_df.sort_values(by=["iteration_id", "time"], inplace=True)
    exp_df["experiment_path"] = exp_path
    
    total_dfs.append(exp_df)

total_df = pd.concat(total_dfs)
total_df

,time,running_objective,current_value,episode_id,iteration_id,v,omega,x_rob,y_rob,vartheta,absolute_path,experiment_path
0,0.000,120.000000,12.000000,1,1,0.003779,0.016373,-1.000000,-1.000000,0.000100,/home/robosrv/huyhoang/regelum-playground/rege...,/home/robosrv/huyhoang/regelum-playground/rege...
1,0.104,119.925564,23.555852,1,1,-0.016573,0.000749,-0.998672,-0.999893,0.009800,/home/robosrv/huyhoang/regelum-playground/rege...,/home/robosrv/huyhoang/regelum-playground/rege...
2,0.207,119.925221,34.694842,1,1,0.037220,0.037756,-0.998649,-0.999896,0.010085,/home/robosrv/huyhoang/regelum-playground/rege...,/home/robosrv/huyhoang/regelum-playground/rege...
3,0.309,119.947945,45.437905,1,1,-0.034477,-0.024547,-0.999233,-0.999893,-0.002355,/home/robosrv/huyhoang/regelum-playground/rege...,/home/robosrv/huyhoang/regelum-playground/rege...
4,0.409,119.944036,55.804204,1,1,-0.048975,-0.004534,-0.999141,-0.999892,-0.001806,/home/robosrv/huyhoang/regelum-playground/rege...,/home/robosrv/huyhoang/regelum-playground/rege...
...,...,...,...,...,...,...,...,...,...,...,...,...
478,49.544,88.931522,289.857776,1,100,-0.004258,0.039569,-0.602709,-0.897079,0.915069,/home/robosrv/huyhoang/regelum-playground/rege...,/home/robosrv/huyhoang/regelum-playground/rege...
479,49.647,89.196336,289.857776,1,100,0.048864,0.019844,-0.603811,-0.898513,0.917324,/home/robosrv/huyhoang/regelum-playground/rege...,/home/robosrv/huyhoang/regelum-playground/rege...
480,49.747,89.107007,289.857776,1,100,-0.002760,0.001319,-0.603369,-0.897926,0.926778,/home/robosrv/huyhoang/regelum-playground/rege...,/home/robosrv/huyhoang/regelum-playground/rege...
481,49.848,88.766070,289.857776,1,100,-0.020097,-0.035788,-0.601928,-0.895996,0.930761,/home/robosrv/huyhoang/regelum-playground/rege...,/home/robosrv/huyhoang/regelum-playground/rege...
